## Importing libraries

In [1]:
import sys
import json
sys.path.insert(0, "./nanohubtools") ## Under develpoment branch
#from nanohubtools import *
import uidl.teleport as t
from uidl.plotly import PlotlyBuilder
from uidl.rappture import RapptureBuilder
from uidl.material import MaterialBuilder
from uidl.material import MaterialContent
from uidl.app import AppBuilder

from uidl.nanohub import Auth
TOOLNAME="nsoptics"
STATE_LOADER_STATUS = "loader_status"
STATE_LOADER_OPEN = "loader_open"
STATE_ERROR_STATUS = "error_status"
STATE_ERROR_OPEN = "error_open"


## Creating the project and adding GUI variables 

In [2]:
Project = t.TeleportProject("Nanosphere Optics Lab")
Component = Project.root
Component.addStateVariable("DrawerIsVisible", {"type":"boolean", "defaultValue": True})
Component.addStateVariable("AppBarStyle", {"type":"string", "defaultValue": "bar_shifted"})
Component.addStateVariable("open_results", {"type":"boolean", "defaultValue": False})
Component.addStateVariable("open_params", {"type":"boolean", "defaultValue": True})

## Getting session information

In [3]:
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]

## Creating Error

In [4]:
ErrorMessage = RapptureBuilder.Error( 
    Component, 
    error_status = STATE_ERROR_STATUS,
    error_open = STATE_ERROR_OPEN,
)

## Creating Loader

In [5]:
Loader = RapptureBuilder.Loader( 
    Component, 
    loader_status = STATE_LOADER_STATUS,
    loader_open = STATE_LOADER_OPEN,
)

## Create Tool Schema

In [6]:
RapptureBuilder.buildSchema(
    Project,
    Component,
    url = "https://nanohub.org/api/tools",
    toolname = TOOLNAME
)
Component.addPropVariable("onLoadSchema", {"type":"func", 'defaultValue' : '(e)=>{e.setState({"'+ STATE_LOADER_OPEN +'":false})}'})    

## Create Session Authentication

In [7]:
Login, CLogin = Auth.Session(
    Project,
    Component,
    sessiontoken = auth_data["sessiontoken"],
    sessionnum = auth_data["sessionnum"],
    url = "https://dev.nanohub.org/api/querycache/squidlog/proxy?url=https://nanohub.org/api/developer/oauth/token",   
)
Login.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": '$e'},
]
Login.content.events["onAuth"] = [
    { "type": "propCall2", "calls": "buildSchema", "args": ['self'] }            
]


In [8]:
Drawer = MaterialBuilder.Drawer(Project, state="DrawerIsVisible", variant="persistent" )

## Defining Input Parameters

In [9]:
SETTINGS = {
    "mat" :    {
        "type":"Select",
        "default_value": "Au-Gold", 
        "units" : "", 
        "options" : {"Au-Gold":"Au-Gold", "Ag-Silver":"Ag-Silver", "Constant":"Constant" },
        "label" : "Particle Composition",
        "description" : '''Choose a material for the particles. The material selection determines the dielectric 
            properties of the nanoparticle. If 'Constant' is selected the material is assumed to be free of loss 
            and the dielectric constant is assumed to be independent of wavelength. ''',
    },
    "cindex" : {
        "type":"Number",
        "default_value": 1.4, 
        "units" : "", 
        "options":{},
        "label" : "Particle Refractive Index", 
        "description" : '''Enter the refractive index of the particle.''', 
        "restrictions" : {"mat":"Constant"}
    },
    "refmed" : {
        "type":"Number",
        "default_value": 1.0, 
        "units" : "", 
        "options":{},
        "label" : "Surrounding Medium Refractive Index",
        "description" : '''Enter the refractive index of the surrounding medium''',
    },
    "radius" : {
        "type":"Number",
        "default_value": 20, 
        "units" : "nm", 
        "options":{},
        "label" : "Radius of particle",
        "description" : '''Radius of particle''',
    },
    "bwavel" : {
        "type":"Number",
        "default_value": 300, 
        "units" : "nm", 
        "options":{},
        "label" : "Beginning wavelength",
        "description" : '''Beginning wavelength''',
    },   
    "ewavel" : {
        "type":"Number",
        "default_value": 1000, 
        "units" : "nm", 
        "options":{},
        "label" : "Ending wavelength",
        "description" : '''Ending wavelength''',
    },
}
#SETTINGS        

## Defining Input Layout

In [10]:
LAYOUT = {
    'input': {
        'type': 'group',
        'id': '',
        'label': '',
        'layout': '',
        'children': [
            { 'type': 'choice', 'id': 'mat', 'label': 'Particle Composition'},
            { 'type': 'number', 'id': 'cindex', 'label': 'Particle Refractive Index', 'enable': [
                {'operand': 'mat', 'operator': '==', 'value': "Constant"}
            ]},
            { 'type': 'number', 'id': 'refmed', 'label': 'Surrounding Medium Refractive Index'},
            { 'type': 'number', 'id': 'radius', 'label': 'Radius of particle'},
            {'type': 'group', 'label': 'wavelength range', 'layout': 'horizontal', 'children': [
                {'type': 'number', 'id': 'bwavel', 'label': 'Beginning wavelength', 'enable': None},
                {'type': 'number', 'id': 'ewavel', 'label': 'Ending wavelength', 'enable': None}
            ]}
        ]
    }
}
#LAYOUT

## Creating the Parameters/Inputs panel

In [11]:
url_sim = "https://dev.nanohub.org/api/querycache/squidlog/proxy?url=https://nanohub.org/api/tools"
NsopticsSettings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url=url_sim,
    toolname = TOOLNAME,
    layout = LAYOUT['input']
)
NsopticsSettings.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False},
    { "type": "stateChange", "modifies": STATE_ERROR_OPEN,"newState": True},
    { "type": "stateChange", "modifies": STATE_ERROR_STATUS,"newState": '$e'},
]
NsopticsSettings.content.events["click"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True}
]
NsopticsSettings.content.events["submit"] = [
    { "type": "stateChange", "modifies": "parameters","newState": '$e.target.value'}
]


## Defining output parameters

In [12]:
RapptureBuilder.loadXY(Project, Component)
RapptureBuilder.loadSequence(Project, Component)
RESULTS = {
    "ESC" : {
        'title' : 'Extinction Cross Section',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['ESC']"] },
    },
    "SCA" : {
        'title' : 'Scattering Cross Section',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['SCA']"] },
    },
    "ABS" : {
        'title' : 'Absortion Cross section',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['ABS']"] },
    },
    "REP" : {
        'title' : 'Real(Dielectric Constant)',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['REP']"] },
    },
    "IEP" : {
        'title' : 'Imaginary(Dielectric Constant)',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['IEP']"] },
    },
}

NsopticsResults = AppBuilder.Results( 
    Component,
    results = RESULTS,
    onClick = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True }],
    onLoad = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False }],
)
NsopticsSettings.content.events["onStatusChange"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_STATUS,"newState": "$e.target.value"}
]
NsopticsSettings.content.events["onSuccess"]=[
  { "type": "stateChange", "modifies": "open_plot","newState":{'ESC':'primary'} },
  { "type": "propCall2", "calls": "loadXY","args": ["self", "['ESC']"]},
  { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False },  
  { "type": "stateChange", "modifies": "open_results","newState": True},
  { "type": "stateChange", "modifies": "open_params","newState": False},
]

## Creating Colorschema and adding nanoHUB logo

In [13]:
ThemeProvider = MaterialBuilder.ThemeProvider( Component, MaterialBuilder.DefaultTheme(
    primary_color = '#699FBB',
    secondary_color = '#f1f1f1',
    primary_bg = '#FFFFFF',
    secondary_bg = '#dbeaf0',
    default_button = 'rgba(255, 255, 255, 0.87)',
    primary_button = 'rgba(255, 255, 255, 0.87)',
    secondary_button = 'rgba(0, 0, 0, 0.87)',
    default_button_bg = 'rgb(63, 162, 192)',
    primary_button_bg = 'rgba(0, 0, 0, 0.65)',
    secondary_button_bg = 'rgba(0, 0, 0, 0.12)',    
))

AppBar = MaterialBuilder.AppBar(
    state="DrawerIsVisible", 
    styles=("AppBarStyle", ["bar_shifted", "bar_normal"]),
    title="Nanosphere Optics Lab"
)

logo = t.TeleportElement(t.TeleportContent(elementType="img"))
logo.content.attrs["width"] = "120"
logo.content.attrs["src"] = "https://nanohub.org/app/site/media/images/PressKit/nanoHUB_logo_color.jpg"
AppBar.content.children[0].addContent(logo)

## Creating Collapsable panels, for input and output parameters 

In [14]:
Project.globals.assets.append({"type": "style", "content": ".bar_shifted {margin-left:350px;}"})
Project.globals.assets.append({ "type": "style", "content": ".bar_normal {margin-left:0px}"})
ExpansionPanel = MaterialBuilder.ExpansionPanel(
    title="Results", 
    content=[NsopticsResults],
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_results"
      }    
    },     
)
ExpansionPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_results","newState": "$toggle"}]
Drawer.addContent(ExpansionPanel)

ParametersPanel = MaterialBuilder.ExpansionPanel(
    title="Parameters", 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_params"
      }    
    }, 
    content=[NsopticsSettings]
)
ParametersPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_params","newState": "$toggle"}]
Drawer.addContent(ParametersPanel)


## Creating Reset button

In [15]:
Gridt = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridt.content.attrs["color"] = "secondary"
Gridt.content.attrs["container"] = True
Gridt.content.attrs["direction"] = "column"
resetSettings = {}
for k,v in SETTINGS.items():
    resetSettings[k] = v["default_value"]

Buttontt= MaterialBuilder.Button(
      title = "Reset Setting", 
      variant = "text", 
      style = {'backgroundColor':'#999999', 'borderRadius' : '0px', 'minHeight':'40px', 'color':'rgba(255, 255, 255, 0.87)'}, 
      onClickButton=[{ "type": "stateChange", "modifies": "parameters","newState": resetSettings}]
)
Gridt.addContent(Buttontt)
Drawer.addContent(Gridt)

## Creating ploting component

In [16]:
BasePlot = PlotlyBuilder.BasePlot( Project, Component)

## Assembling the app

In [17]:
Gridh = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridh.content.attrs["container"] = True
Gridh.content.attrs["direction"] = "row" 
Gridh.addContent(Drawer)
Gridh.addContent(BasePlot)  

Gridv = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridv.content.attrs["container"] = True
Gridv.content.attrs["direction"] = "column"
Gridv.addContent(AppBar)
Gridv.addContent(Gridh)


ThemeProvider.addContent(Gridv)
ThemeProvider.addContent(Loader)
ThemeProvider.addContent(ErrorMessage)
ThemeProvider.addContent(Login)

Component.addNode(ThemeProvider)
Project.buildReact( TOOLNAME+auth_data["sessiontoken"]+".html");

## Displaying the app in jupyter

In [18]:
from IPython.core.display import display
from IPython.display import IFrame
display(IFrame(src=TOOLNAME+auth_data["sessiontoken"]+".html", width="100%", height="800"))